In [1]:
import ast
from typing import Annotated, TypedDict

from langchain_community.tools import DuckDuckGoSearchRun
from langchain_core.messages import HumanMessage
from langchain_core.tools import tool
from langchain_ollama import ChatOllama
from langgraph.graph import START, StateGraph
from langgraph.graph.message import add_messages
from langgraph.prebuilt import ToolNode, tools_condition

/home/anseh/.pyenv/versions/langchain/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [ ]:
@tool
def calculator(query: str) -> str:
    """계산기, 수식만 입력받습니다."""
    return ast.literal_eval(query)


search = DuckDuckGoSearchRun()
tools = [search, calculator]
model = ChatOllama(model="granite3.3:2b", temperature=0.1).bind_tools(tools)

In [ ]:
class State(TypedDict):
    messages: Annotated[list, add_messages]


def model_node(state: State) -> State:
    res = model.invoke(state["messages"])
    return {"messages": res}


builder = StateGraph(State)
builder.add_node("model", model_node)
builder.add_node("tools", ToolNode(tools))
builder.add_edge(START, "model")
builder.add_conditional_edges("model", tools_condition)
builder.add_edge("tools", "model")

graph = builder.compile()

In [ ]:
input = {
    "messages": [HumanMessage("미국의 제30대 대통령이 사망했을 때 몇 살이었나요?")]
}

for c in graph.stream(input):
    print(c)

{'model': {'messages': AIMessage(content='', additional_kwargs={}, response_metadata={'model': 'granite3.3:2b', 'created_at': '2025-11-03T00:59:40.062155787Z', 'done': True, 'done_reason': 'stop', 'total_duration': 3265954859, 'load_duration': 80961090, 'prompt_eval_count': 262, 'prompt_eval_duration': 186531292, 'eval_count': 46, 'eval_duration': 2996423947, 'model_name': 'granite3.3:2b'}, id='run--5f059300-7574-4a4e-86f5-231c43d12649-0', tool_calls=[{'name': 'duckduckgo_search', 'args': {'query': '제30대 미국 대통령 사망 후 몇 살이 있었는지'}, 'id': '25093a42-7b0d-4dcb-8942-23fcd37dbb6d', 'type': 'tool_call'}], usage_metadata={'input_tokens': 262, 'output_tokens': 46, 'total_tokens': 308})}}
{'tools': {'messages': [ToolMessage(content="대통령직 계승권을 갖기 위해선 미국 태생이면서 35 세 이상이며, 미국 내에 최소한 14년 이상을 거주한 시민권자여야 한다. 1841년 취임 31일 만에 사망 한 윌리엄 헨리 해리슨 이 미국 역사상 가장 짧은 대통령 임기를 가졌으며, [6] 프랭클린 D. 루스벨트 는 1933년부터 1945년 4번째 임기 초반에 사망 할 때까지 12년이 넘는 가장 긴 대통령 임기를 가졌다. Dec 6, 2024 · 평가: 위기 동안의 리더십으로 인해 가장 위대한 미국 대통령 중 하나로 널리 인정